In [1]:
import torch
import os
import numpy as np

from helpers import vis, get_all_frames_labels
from mae import mae_vit_base_patch16_dec512d8b

os.chdir('../aot-benchmark')

%load_ext autoreload
%autoreload 2

/home/tpeat3/.conda/envs/aot38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'util'

In [5]:
device='cuda'

In [6]:
model = mae_vit_base_patch16_dec512d8b(img_size=465).to(device)

In [7]:
# load dataset

import importlib
from dataloaders.train_datasets import AOT_Train
from torch.utils.data import DataLoader
import dataloaders.video_transforms as tr
from torchvision import transforms

stage = 'pre'
exp_name = "tristan_test"
model_name = "aott"
engine_config = importlib.import_module('configs.' + stage)
cfg = engine_config.EngineConfig(exp_name, model_name)

cfg.DIST_START_GPU = 0  # default value
cfg.TRAIN_GPUS = 1
cfg.DATASETS = 'AOT'
cfg.DIR_AOT = '/gv1/projects/AI_Surrogate/dev/tristan/aot-benchmark/datasets/AOT'
cfg.PRETRAIN_MODEL = "pretrain_model/mobilenet_v2-b0353104.pth"
cfg.DIST_ENABLE = False
cfg.DATA_WORKERS = 2
cfg.TRAIN_BATCH_SIZE = 1

cfg.enable_prev_frame = True
train_sampler = None

composed_transforms = transforms.Compose([
                tr.RandomScale(cfg.DATA_MIN_SCALE_FACTOR,
                               cfg.DATA_MAX_SCALE_FACTOR,
                               cfg.DATA_SHORT_EDGE_LEN),
                tr.BalancedRandomCrop(cfg.DATA_RANDOMCROP,
                                      max_obj_num=cfg.MODEL_MAX_OBJ_NUM),
                tr.RandomHorizontalFlip(cfg.DATA_RANDOMFLIP),
                tr.Resize(cfg.DATA_RANDOMCROP, use_padding=True),
                tr.ToTensor()])

train_dataset = AOT_Train(
                root=cfg.DIR_AOT,
                transform=composed_transforms,
                seq_len=cfg.DATA_SEQ_LEN,
                rand_gap=cfg.DATA_RANDOM_GAP_AOT,
                rand_reverse=cfg.DATA_RANDOM_REVERSE_SEQ,
                merge_prob=cfg.DATA_DYNAMIC_MERGE_PROB,
                enable_prev_frame=cfg.enable_prev_frame,
                max_obj_n=cfg.MODEL_MAX_OBJ_NUM)

train_loader = DataLoader(train_dataset,
               batch_size=int(cfg.TRAIN_BATCH_SIZE /
                              cfg.TRAIN_GPUS),
               shuffle=False if cfg.DIST_ENABLE else True,
               num_workers=cfg.DATA_WORKERS,
               pin_memory=True,
               sampler=train_sampler,
               drop_last=True,
               prefetch_factor=4)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~
./result/tristan_test_AOTT/PRE/ckpt
*********************************************************************************************************************************
Video Num: 23 X 1


In [8]:
for frame_idx, sample in enumerate(train_loader):
    all_frames, all_labels, bs, obj_nums = get_all_frames_labels(sample, 0) # obj nums is the classes
    break

In [13]:
all_frames.shape

torch.Size([96, 3, 465, 465])

In [14]:
obj_nums

[2, 2, 1, 0, 2, 2, 2, 0, 2, 2, 1, 2, 2, 2, 1, 2]

In [9]:
model(all_frames)

torch.Size([96, 841, 768])
torch.Size([1, 842, 768])


RuntimeError: CUDA out of memory. Tried to allocate 4.06 GiB (GPU 0; 31.75 GiB total capacity; 18.86 GiB already allocated; 1.09 GiB free; 18.99 GiB reserved in total by PyTorch)